In [ ]:
import pandas as pd
import utils

In [ ]:
X_label_encoded=pd.read_csv('Data//X_label_encoded.csv')

X = X_label_encoded.drop(columns=['fog_train.class'])
y = X_label_encoded['fog_train.class']

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from flaml import AutoML
import seaborn as sns
import matplotlib.pyplot as plt

# 훈련 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# AutoML 모델 정의 및 학습 (일부 데이터로 확인)
automl = AutoML()

# 전체 데이터로 학습
automl.fit(X_train, y_train, task='classification', time_budget=600, log_file_name="automl.log")

# 최종 모델 예측
y_pred = automl.predict(X_test)

# 성능 평가
print(classification_report(y_test, y_pred))

# 다중 CSI 계산
csi = utils.calculate_csi(y_test, y_pred)

print(f'CSI: {csi}')

# 혼동 행렬 시각화
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
'''
CSI: 0.06412213740458016
'''

In [ ]:
import utils
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

In [ ]:
# H2O 클러스터 시작
h2o.init()

# 훈련 및 테스트 데이터 분리
X_train_le, X_test_le, y_train, y_test = train_test_split(X_label_encoded, y, test_size=0.2, random_state=42)
X_train_ohe, X_test_ohe, _, _ = train_test_split(X_ohe_encoded, y, test_size=0.2, random_state=42)

# H2O 프레임으로 변환 (레이블 인코딩된 데이터)
train_le = h2o.H2OFrame(pd.concat([X_train_le, y_train], axis=1))
test_le = h2o.H2OFrame(pd.concat([X_test_le, y_test], axis=1))

# H2O 프레임으로 변환 (원-핫 인코딩된 데이터)
train_ohe = h2o.H2OFrame(np.column_stack((X_train_ohe, y_train)))
test_ohe = h2o.H2OFrame(np.column_stack((X_test_ohe, y_test)))

# H2O AutoML 실행 (레이블 인코딩된 데이터)
aml_le = H2OAutoML(max_runtime_secs=3600, seed=42)
aml_le.train(y='target', training_frame=train_le)

# H2O AutoML 실행 (원-핫 인코딩된 데이터)
aml_ohe = H2OAutoML(max_runtime_secs=3600, seed=42)
aml_ohe.train(y=train_ohe.ncol-1, training_frame=train_ohe)

# 예측 (레이블 인코딩된 데이터)
preds_le = aml_le.leader.predict(test_le)
y_pred_le = preds_le.as_data_frame().iloc[:, 0].values

# 예측 (원-핫 인코딩된 데이터)
preds_ohe = aml_ohe.leader.predict(test_ohe)
y_pred_ohe = preds_ohe.as_data_frame().iloc[:, 0].values

# 다중 CSI 계산 및 출력
csi_le = utils.calculate_csi(y_test, y_pred_le)
csi_ohe = utils.calculate_csi(y_test, y_pred_ohe)

print(f'Label Encoded Data - CSI: {csi_le}')
print(f'One-Hot Encoded Data - CSI: {csi_ohe}')

# H2O 클러스터 종료
h2o.shutdown()